In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets
license_plate_model = YOLO('license_plate_detector.pt')  # Model for license plate detection

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output22.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps object centers to unique IDs
previous_positions = {}  # Stores the last known positions of objects
previous_frames = {}  # Stores the frame numbers for speed calculation
max_speed_dict = {}  # Stores the maximum speed detected for each object
printlist={}
helmet_status_dict = {} 
next_id = 1  # Counter for assigning unique IDs

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}  # COCO class IDs for vehicles

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.01  # Conversion factor from pixel distance to meters

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    ret, frame = cap.read()
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if not ret:
        break

    current_centers = {}

    # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1
                
            if unique_id not in printlist.keys():
                printlist.update(unique_id,['x',-1,-1])
            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            # Store the maximum speed detected
            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

            # Update previous positions and frames
            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            # Draw bounding box and label
            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]} {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                if unique_id not in printlist.keys():
                    printlist.update(unique_id,['y',-1,-1])
                
                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Speed Calculation
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                # Default label to "Unknown" if no helmet status is detected yet
                helmet_status = helmet_status_dict.get(unique_id, "Unknown")
                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h {helmet_status}"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                helmet_status = "Helmet" if class_id == 2 else "No Helmet"

                # Associate helmet status with the closest driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)
                unique_id = get_closest_id(center, object_ids)
                if unique_id is not None:
                    helmet_status_dict[unique_id] = helmet_status

                cv2.putText(frame, helmet_status, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)


    # Detect license plates
    license_plate_results = license_plate_model.predict(frame, stream=True)
    for lp_result in license_plate_results:
        boxes = lp_result.boxes
        for lp_box in boxes:
            lp_x1, lp_y1, lp_x2, lp_y2 = map(int, lp_box.xyxy[0])
            label = "License Plate"
            cv2.rectangle(frame, (lp_x1, lp_y1), (lp_x2, lp_y2), (255, 0, 0), 1)  # Blue for license plates
            cv2.putText(frame, label, (lp_x1, lp_y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    object_ids = current_centers
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")
